## Wrangle and Analyze Data

In this project a dataset from Twitter profile WeRateDogs will be wrangled (gather, assess and clean). All the efforts for this task are going to be reported in this notebook. 

### Name: Daniel Guarino

## Table of Contents

- [Introduction](#intro)
- [Basic Libraries](#libraries)
- [Part I - Gathering Data](#gathering)
- [Part II - Assessing Data](#assess)
- [Part III - Cleaning Data](#regression)


<a id='intro'></a>
### Introduction

Wrangling data is very important.....

<a id='libraries'></a>
### Import Basic Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

<a id='gathering'></a>
### Part I - Gathering Data

**Reading CSV file (manually downloaded from 'Resources' folder on Udacity - Wrangle and Analyze Data web page)**

In [2]:
df = pd.read_csv('twitter_archive/twitter-archive-enhanced.csv')

In [3]:
df.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


**Downloading a file programmatically**

In [4]:
import requests
import os

In [5]:
folder_name = 'twitter_archive'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
response

<Response [200]>

In [6]:
with open(os.path.join(folder_name, 
                       url.split('/')[-1]), mode = 'wb') as file:
    file.write(response.content)

In [7]:
os.listdir(folder_name)

['twitter-archive-enhanced.csv', 'image-predictions.tsv', 'tweet_json.txt']

In [8]:
df_image = pd.read_csv('twitter_archive/image-predictions.tsv', sep='\t')
df_image.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


**Importing and accessing Twitter API (tweep)**

In [9]:
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

In [11]:
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True)

In [12]:
tweet_ids = df.tweet_id.values
len(tweet_ids)

2356

In [ ]:
# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('twitter_archive/tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

In [10]:
# Converting relevant JSON data to dataframe

tweets_list =[]

with open('twitter_archive/tweet_json.txt') as json_file:
    for line in json_file:
    
        tweets_dict = {}
        tweets_json = json.loads(line)
        
        try:
            tweets_dict['tweet_id'] = tweets_json['extended_entities']['media'][0]['id']
        except:
            tweets_dict['tweet_id'] = 'na'

        tweets_dict['retweet_count'] = tweets_json['retweet_count']
        tweets_dict['favorite_count'] = tweets_json['favorite_count']
        
        tweets_list.append(tweets_dict)


In [11]:
df_api = pd.DataFrame(tweets_list)
df_api.head()

,tweet_id,retweet_count,favorite_count
0,892420639486877696,7947,37149
1,892177413194625024,5902,31957
2,891815175371796480,3902,24066
3,891689552724799489,8099,40478
4,891327551943041024,8785,38706


<a id='assess'></a>
### Part II - Assessing Data

Main data frame info

Image data frame info

Json file info

**Data Quality Analyzis**